### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [179]:
%%time
from collections import namedtuple
import random
import numpy as np
from collections import deque
import time
import sys
from decimal import *

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000): #10000000
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    result[3] = result[3] / 1000 # делим только сейчас на количество элементов чтобы найти среднее
    result[4] = float(result[4]) # переводим из Decimal во float необходимо
    return ['{:0.2f}'.format(x) for x in result / count]

def get_features(stream):
    sorted_array = list() # создаем пустой массив для хранения сортированных значений элементов окна
    window = deque() # создаем дек(структура данных) для хранения очередности добавления элементов. стоит почитать если не знаешь что это. быстро извлекает первое и последнее значение, но все остальное медленно поэтому массив вверху создали
    sum_of_array = 0 # инициализируем переменную которая будет хранить среднее значение окна, но не деленное на кол-во, будем делать в конце
    flag = False # булевая переменная, ждем пока окно до тысячи первых значений не наберется
    variance = 0 # инициализируем дисперсию
    sum_of_array_old = 0 # необходимо помнить среднее предыдущего окна для вычисления дисперсии реккурентной
    for value in stream: # считываем каждое значение и потока
        sorted_array.append(value) # добавляем новое значение в массив, нужен потому что быстрее сортировка происходит, чем в деке
        window.append(value) # добавляем новое значение в дек (нужен для запоминания как именно добавлялись элементы)
        sum_of_array += value # добавляем каждый элемент чтобы посчитать среднее занчение
        if flag: # обработка каждого окна если добрались до количества тысячи
            last_element = window.popleft() # вытаскиваем самый старый элемент и удаляем
            sum_of_array -= last_element # убираем его из среднего 
            sorted_array.remove(last_element) # удаляем из массива этот элемент
            sorted_array.sort() # теперь сортируем массив 
            """
            хитрожопое вычисление дисперсии с помощью реккурентной формулы, зная предыдущее значение дисперсии окна, 
            выводится математически, считается так сложно, потому что иначе будет очень долго, чем обычное np.var()
            """
            reccurent_element = Decimal((value - last_element) * (value + last_element - ((sum_of_array + sum_of_array_old) / 1000)) / 1000)
            variance = variance + reccurent_element 
            sum_of_array_old = sum_of_array # запоминаем среднее значение для следующей итерации
            """
            # в отсортированном массиве самый маленький эл всегда первый, самый большой последний,
            медиана всегда в середине, также возвращаем остальные посчитанные значения
            """
            yield sorted_array[0], sorted_array[999], sorted_array[500], sum_of_array, variance 
        if len(sorted_array) == 1000 and flag == False: # здесь ждем момент когда будет первая тысяча значений, исполняется разово
            flag = True # устанавливаем флаг тру что первую тысячу собрали, больше в этот кусок кода никогда не зайдет
            sorted_array.sort() # теперь сортируем массив 
            sum_of_array_old = sum_of_array 
            variance = np.var(sorted_array, dtype=np.float64) #первый раз считаем по обычному сложному пути дисперси, потом можно реккуернтно
            variance = Decimal.from_float(variance) # нужно перевести в тип данных Decimal из float потому что важна точность
            yield sorted_array[0], sorted_array[999], sorted_array[500], sum_of_array, variance

#вспомогательная функция которая выводит фичи окна можно вызвать чтобы узнать что внутри каждого окна            
def print_features():
    counter = 0
    for i in get_features(easy_stream()):
        counter = counter + 1
        print(i, counter)
        

#print(get_tuple_stream_mean(get_features(easy_stream()), 5))
print(get_tuple_stream_mean(get_features(medium_stream()), 5))
#print(get_tuple_stream_mean(get_features(nightmare_stream()), 5))
#print_features()

['0.02', '254.98', '127.60', '127.48', '5455.17']
['1017512.29', '999017359.97', '500438415.64', '499880345.88', '83228908564031120.00']
CPU times: user 17min 33s, sys: 5.61 s, total: 17min 38s
Wall time: 17min 56s
